In [1]:
import numpy as np

In [43]:
# algo espilon greedy (exploration vs exploitation) 
def algo_e(Q,state,epsilon,all_actions):
    if np.random.rand() < epsilon:
        action=np.random.choice(all_actions) #random action
    else:
        action=np.argmax(Q[state]) #with the highest q_value
    return action

#to change for UCB algo

In [93]:
data=pd.read_csv(f"data/slice{0}.csv", index_col = 0)
data.head(10)

,0,1,2,3,4,5,6,7,8,9,...,166,167,168,169,170,171,172,173,174,175
0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.64,0.64,0.68,0.56,0.60,0.60,0.60,0.64,0.52,0.56
1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.60,0.68,0.64,0.64,0.64,0.56,0.64,0.60,0.60,0.60
2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.56,0.56,0.68,0.64,0.60,0.52,0.52,0.64,0.60,0.56
3,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.52,0.44,0.44,0.52,0.48,0.48,0.44,0.44,0.52,0.48
4,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.68,0.56,0.52,0.56,0.56,0.64,0.52,0.48,0.52,0.52
5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.60,0.60,0.60,0.56,0.56,0.56,0.56,0.56,0.52,0.56
6,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.56,0.64,0.68,0.48,0.48,0.56,0.60,0.64,0.48,0.48
7,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.72,0.60,0.48,0.56,0.48,0.68,0.56,0.48,0.52,0.48
8,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.48,0.68,0.60,0.48,0.48,0.48,0.64,0.56,0.48,0.48
9,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.60,0.56,0.52,0.64,0.56,0.56,0.52,0.52,0.60,0.52


In [49]:
def peak_one(data):
    row = data.sample(n=1).iloc[0]
    x1 = row[:52].values
    x2 = row[52:104].values
    c = row[104:].values
    return x1, x2, c

#example
x1, x2, c = peak_one(data)
print("x1:", x1)
print("x2:", x2)
print("c:", c)

x1: [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0.
 0. 1. 0. 0.]
x2: [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 1.]
c: [0.88 0.8  1.   0.96 0.88 0.88 0.76 0.84 0.84 0.84 0.84 0.72 0.8  0.76
 0.76 0.76 0.68 0.76 0.72 0.72 0.72 0.64 0.68 0.68 0.68 0.68 0.6  0.64
 0.64 0.64 0.64 0.56 0.6  0.6  0.6  0.6 ]
<class 'numpy.ndarray'>


In [35]:
# Number of layers
l = 5

# Initialize action-value function Qj with random weights for j = 1, ..., l
Q = [np.random.rand(52, 52, 36) for _ in range(l)]

# Print the shape of Qj to verify
for j in range(l):
    print(f"Q{j+1} shape: {Q[j].shape}")

Q1 shape: (52, 52, 36)
Q2 shape: (52, 52, 36)
Q3 shape: (52, 52, 36)
Q4 shape: (52, 52, 36)
Q5 shape: (52, 52, 36)


In [44]:
j=3
hand = 0 if j % 2 == 1 else 0  
print(hand)


0


In [ ]:
all_actions=list(range(36))
nb_layers=5
#state=[hand,history_bidding]

def learning_algo(data,nb_layers=5,epsilon=0.1):
  D=[]
  Q_models = [np.random.rand(52, 52, 36) for _ in range(nb_layers)]
  
  #repeat avec early stopping
    layers_explored=0
    bidding_history=[]
    x1,x2,scores=peak_one(data)

    for j in range(nb_layers):
      C=np.zeros(36)

      if j%2==1:
        hand=x1
      else:
        hand=x2

      for i,action in enumerate(all_actions):
        C[i]=algo_p(action,scores)

      D.append([(hand,bidding_history.copy()),C])
      
      state = (np.argmax(hand), np.argmax(bidding_history) if bidding_history else 0)

      next_a=algo_e(Q_models[j],state,epsilon,all_actions)

      if next_a=="PASS":
        layers_explored=j
        break
      bidding_history.append(next_a)

    for i in range(np.max(layers_explored,nb_layers)):
      #a voir si minibatch ou pas, peut commencer sans
      train(Q,D)

In [ ]:
def train(Q, D, learning_rate=0.01, gamma=0.9):
    for (state, C) in D:
        hand, history = state
        state_idx = (np.argmax(hand), np.argmax(history) if history else 0)
        
        for action, reward in enumerate(C):
            target = reward + gamma * np.max(Q[state_idx])  # Mise à jour des Q-values
            Q[state_idx[0], state_idx[1], action] += learning_rate * (target - Q[state_idx[0], state_idx[1], action])

# Result


In [149]:
import numpy as np
import pandas as pd
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from nn_tools import Qfirst, Qfollowing, one_epoch_layers,optimiers_list


In [52]:
bidding_history=np.zeros(36)
bidding_history[14:17]=1
num_ones = np.count_nonzero(bidding_history == 1)
print(f"Number of 1s in bidding_history: {num_ones}")

Number of 1s in bidding_history: 3


# Algo_p

### Brouillon

In [ ]:
# Déterminer le coût d'une action en fonction des scores
def algo_p(action,x1,x2,bidding_history,scores,Q_models):
    num_ones = np.count_nonzero(bidding_history == 1)
    for i in range(num_ones,nb_layers):
        if i % 2 == 1:
            hand = x1
        else:
            hand = x2
        state=hand if i==0 else (hand,bidding_history)

        next_a=np.argmax(Q_models[i][state][action]) 
        bidding_history[next_a]=1

        if next_a==len(bidding_history)-1 or i==nb_layers-1:
            last_action=next_a

        action=next_a

    return scores[last_action]

In [ ]:
import numpy as np
import torch

def algo_p(action, x1, x2, bidding_history, scores, Q_models, nb_layers):
    num_ones = np.count_nonzero(bidding_history == 1)

    for i in range(num_ones, nb_layers):
        hand = x1 if i % 2 == 1 else x2  # Détermine la main en fonction du joueur actif
        state = torch.tensor(hand, dtype=torch.float32) if i == 0 else torch.tensor(np.concatenate([hand, bidding_history]), dtype=torch.float32)

        # Passer l'état dans le modèle Q et choisir l'action avec la plus grande valeur Q
        with torch.no_grad():
            q_values = Q_models[i](state)  # Prédiction du modèle Q
            next_a = torch.argmax(q_values).item()  # Récupérer l'action avec le Q maximal

        bidding_history[next_a] = 1  # Met à jour l'historique des enchères

        if next_a == len(bidding_history) - 1 or i == nb_layers - 1:
            last_action = next_a  # Sauvegarde de la dernière action valide

        action = next_a  # Mise à jour de l'action pour la prochaine itération

    return scores[last_action]


### Version finale

In [167]:

def perform_bids(x1, x2, scores,bidding_history, Q_models, nb_layers):
    num_ones = np.count_nonzero(bidding_history == 1)

    for i in range(num_ones, nb_layers):
        hand = x1 if (i+1) % 2 == 1 else x2  # Détermine la main en fonction du joueur actif
        
        state = torch.tensor(hand, dtype=torch.float32) if i == 0 else torch.tensor(np.concatenate([hand, bidding_history]), dtype=torch.float32)

        # passer l'état dans le modèle Q[i] et obtenir les valeurs Q[i]
        with torch.no_grad():
            q_values = Q_models[i](state)  # prédiction du modèle Q[i] pour le state

            next_a = torch.argmax(q_values).item()  # sélection greedy max(Q)   a mettre la regle next_a>derniere enchere

        bidding_history[next_a] = 1  # met à jour l'historique des enchères

        if next_a == len(bidding_history) - 1 or i == nb_layers - 1:
            last_action = next_a  # sauvegarde de la dernière action valide
 
    return scores[last_action]


In [107]:

Q_models = [Qfirst() if i == 0 else Qfollowing() for i in range(nb_layers)]

perform_bids(x1, x2, c, bidding_history, Q_models, 5)

hand: [0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0.
 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
state: tensor([0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0.,
        0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
next_a: 5
bidding_history: [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


hand: [1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
 0. 1. 0. 0.]
state: tensor([1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0

np.float64(0.76)

In [164]:
def algo_p(action, x1, x2, scores,bidding_history, Q_models, nb_layers):
    updated_bid_history = bidding_history.copy()
    updated_bid_history[action] = 1
    if np.count_nonzero(updated_bid_history) == nb_layers:
        return scores[action]
    final_score=perform_bids(x1, x2,scores, updated_bid_history, Q_models, nb_layers)
    return final_score

In [113]:
#example
x1, x2, scores = peak_one(data)
print("x1:", x1)
print("x2:", x2)
print("scores:", scores)
bidding_history=np.zeros(36)
bidding_history[5]=1
action=6

nb_layers = 5
Q_models = [Qfirst() if i == 0 else Qfollowing() for i in range(nb_layers)]

algo_p(action,x1,x2,scores,bidding_history,Q_models,nb_layers)

x1: [0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0.
 1. 0. 0. 0.]
x2: [0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1.
 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.
 0. 0. 0. 0.]
scores: [0.84 0.84 0.96 1.   0.88 0.92 0.8  0.96 1.   0.76 0.8  0.72 0.96 0.84
 0.72 0.76 0.68 0.76 0.76 0.68 0.68 0.64 0.72 0.72 0.64 0.64 0.6  0.68
 0.68 0.6  0.6  0.56 0.64 0.64 0.56 0.6 ]
hand: [0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0.
 1. 0. 0. 0.]
state: tensor([0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0.,
        1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

np.float64(0.72)

# Algo_E et peak_one

In [105]:
# Stratégie epsilon-greedy pour explorer/exploiter les actions
def algo_e(Q, state, epsilon, all_actions):
    if np.random.rand() < epsilon:
        return np.random.choice(all_actions)
    else:
        with torch.no_grad():
            print(Q(state))
            return np.argmax(Q(state))  

# Sélectionne une ligne aléatoire du dataset et extrait les informations
def peak_one(data):
    row = data.sample(n=1).iloc[0]  # Sélection aléatoire d’une ligne
    x1 = row[:52].values  # Main du joueur 1
    x2 = row[52:104].values  # Main du joueur 2
    scores = row[104:140].values  # Scores des 36 actions possibles joueur
    return x1, x2, scores

In [98]:
x1,x2,c1,c2=peak_one(data)
print("x1:", x1)
print("x2:", x2)
print("c1:", c1)    
print("c2:", c2)

x1: [0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0.
 0. 1. 0. 0.]
x2: [0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0.]
c1: [1.   0.68 0.68 0.76 0.72 0.68 0.64 0.64 0.72 0.68 0.64 0.6  0.6  0.68
 0.64 0.6  0.6  0.56 0.64 0.6  0.56 0.56 0.56 0.6  0.56 0.56 0.52 0.52
 0.56 0.52 0.52 0.52 0.52 0.56 0.52 0.52]
c2: [1.   0.68 0.68 0.76 0.72 0.68 0.64 0.64 0.72 0.68 0.64 0.6  0.6  0.68
 0.64 0.6  0.6  0.56 0.64 0.6  0.56 0.56 0.56 0.6  0.56 0.56 0.52 0.52
 0.56 0.52 0.52 0.52 0.52 0.56 0.52 0.52]


In [145]:
def update(D, state, C, j):
    D[j][0] = torch.concat([D[j][0], state.clone().detach().unsqueeze(0)])
    D[j][1] = torch.concat([D[j][1], torch.tensor(C, dtype=torch.float32).unsqueeze(0)])
    return D

# Main 

In [168]:
# Entraînement du modèle spécifique à chaque couche d'enchère
def learning_algo(data,max_iter=100, nb_layers=5, epsilon=0.1,
                  loss= nn.MSELoss(),batch_size=50, device='cpu'):

    D = [[torch.empty(0), torch.empty(0)] for _ in range(nb_layers)]  # Un dataset D par couche
    all_actions = list(range(36))  # 36 actions possibles
    Q_models = [Qfirst() if i == 0 else Qfollowing() for i in range(nb_layers)]
    losses=[]
    for _ in range(max_iter):
        bidding_history = np.zeros(36) # array de 5x36 pour enregistrer l'historique des enchères
        x1, x2, scores = peak_one(data)

        for j in range(nb_layers):
            C = np.zeros(36)  
            hand = x1 if (j+1) % 2 == 1 else x2  # main du joueur actuel

            # Calcul des coûts des actions
            for i, action in enumerate(all_actions):
                C[i] = algo_p(action,x1,x2,scores,bidding_history,Q_models,nb_layers)
            
            # state = main si première couche sinon state = main + enchère_précédente
            state = torch.tensor(hand, dtype=torch.float32) if j == 0 else torch.tensor(np.concatenate([hand, bidding_history]), dtype=torch.float32)

            D=update(D,state,C,j) #ajout de l'expérience dans le dataset D_j
            
            next_a = algo_e(Q_models[j], state, epsilon, all_actions)  # Sélection avec Q_j
            
            if next_a == len(bidding_history) - 1:  # si l'action est PASS
                break  
            
            bidding_history[next_a] = 1  # enregistrement de l'action choisie dans l'historique des enchères


        # Entraînement des modèles Q
        opti_list=optimiers_list(Q_models)
        losses.append(one_epoch_layers(Q_models, D, opti_list,loss, batch_size, device))
    return Q_models
    

# Prendre l'entrainement de Pierrot
# Fonction d'entraînement pour chaque modèle Q
# def train(Q, D, learning_rate=0.01, gamma=0.9):
#     for state, C in D:
#         for action, reward in enumerate(C):
#             target = reward + gamma * np.max(Q[state])  # Mise à jour de Bellman
#             Q[state[0], state[1], action] += learning_rate * (target - Q[state[0], state[1], action])


In [169]:
learning_algo(data,max_iter=2, nb_layers=5, epsilon=0.1,
                  loss= nn.MSELoss(),batch_size=50, device='cpu')

tensor([-0.0612,  0.0036,  0.1176,  0.0202,  0.0113, -0.0168,  0.0635,  0.1179,
         0.0176,  0.0607,  0.0193,  0.0267, -0.0052, -0.0171,  0.0397,  0.0137,
        -0.0374,  0.0962,  0.0575,  0.0007, -0.0104, -0.0762,  0.0633,  0.0479,
         0.0151, -0.0654, -0.0236, -0.0588,  0.0456, -0.0546, -0.0834,  0.0248,
        -0.0857, -0.1218,  0.0488, -0.1065])
tensor([-7.7297e-02,  5.3583e-02, -8.6979e-02, -3.0479e-02, -7.9605e-02,
        -2.0148e-02, -6.2806e-02, -4.6856e-02,  4.0937e-02,  2.3690e-02,
        -8.2505e-02,  6.7428e-02,  1.2974e-02,  1.5879e-02, -6.0959e-02,
        -1.2993e-02,  8.4731e-02, -3.4982e-02, -8.5863e-02,  6.8263e-02,
        -2.3658e-02, -4.6052e-02, -9.1390e-03,  2.9613e-02, -2.3540e-02,
         1.4946e-03, -9.5212e-03, -3.2355e-02, -1.6635e-03,  6.4065e-02,
        -2.7838e-02,  5.6802e-02, -5.9784e-02, -4.1441e-02, -5.6408e-02,
        -3.6731e-06])
tensor([-0.1229,  0.0887,  0.0391,  0.0598,  0.0187,  0.1222, -0.0415, -0.0509,
         0.0691, -0.07

AttributeError: partially initialized module 'torch._dynamo' has no attribute 'external_utils' (most likely due to a circular import)

### brouillon

In [139]:
torch.tensor(C, dtype=torch.float32).unsqueeze(0)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [143]:
D = [[torch.empty(0), torch.empty(0)] for _ in range(5)]
D = update(D, torch.tensor([1, 2, 3], dtype=torch.float32), [0.1, 0.2, 0.3], 0)
D = update(D, torch.tensor([1, 2, 3], dtype=torch.float32), [0.1, 0.2, 0.3], 0)
print(D[0][0])
print(D[0][1])

tensor([[1., 2., 3.],
        [1., 2., 3.]])
tensor([[0.1000, 0.2000, 0.3000],
        [0.1000, 0.2000, 0.3000]])


## Test avec chat V1

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import random

In [6]:
def create_model(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(output_dim, activation='linear'))  # Q-value output for each action
    model.compile(loss='mse', optimizer=Adam(lr=0.001))
    return model

In [7]:
# Define the epsilon-greedy strategy for action selection
def epsilon_greedy(model, state, epsilon, action_space):
    if random.random() < epsilon:
        return random.choice(action_space)  # Choose a random action (exploration)
    else:
        q_values = model.predict(state)  # Predict Q-values for each action
        return np.argmax(q_values[0])  # Choose the action with the highest Q-value (exploitation)


In [8]:
def train_drl_model(data, model, gamma=0.9, epsilon=0.1, batch_size=64, epochs=10):
    # data: [(x1, x2, contract_scores)] - where contract_scores is a vector of scores for possible actions
    action_space = len(data[0][2])  # The number of possible actions (contracts)
    
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        for i in range(len(data)):
            x1, x2, contract_scores = data[i]
            state = np.array([x1, x2]).reshape(1, -1)
            target = model.predict(state)  # Get current Q-values for the state
            
            # Select the action (bid) using epsilon-greedy
            action = epsilon_greedy(model, state, epsilon, range(action_space))
            
            # Compute the reward (contract score) for the chosen action
            reward = contract_scores[action]  # Get the score for the chosen action
            
            # Update the Q-value using the Bellman equation
            target[0][action] = reward + gamma * np.max(target[0])  # Bellman update
            
            # Train the model on this updated Q-value
            model.fit(state, target, epochs=1, verbose=0)
        
        epsilon = max(0.01, epsilon * 0.995)  # Decay epsilon to reduce exploration over time


In [17]:
data=pd.read_csv(f"data/slice{0}.csv", index_col = 0)

In [19]:
data=data.iloc[:,:140]
data.head(10)

,0,1,2,3,4,5,6,7,8,9,...,130,131,132,133,134,135,136,137,138,139
0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.64,0.64,0.68,0.56,0.60,0.60,0.60,0.64,0.52,0.56
1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.60,0.68,0.64,0.64,0.64,0.56,0.64,0.60,0.60,0.60
2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.56,0.56,0.68,0.64,0.60,0.52,0.52,0.64,0.60,0.56
3,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.52,0.44,0.44,0.52,0.48,0.48,0.44,0.44,0.52,0.48
4,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.68,0.56,0.52,0.56,0.56,0.64,0.52,0.48,0.52,0.52
5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.60,0.60,0.60,0.56,0.56,0.56,0.56,0.56,0.52,0.56
6,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.56,0.64,0.68,0.48,0.48,0.56,0.60,0.64,0.48,0.48
7,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.72,0.60,0.48,0.56,0.48,0.68,0.56,0.48,0.52,0.48
8,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.48,0.68,0.60,0.48,0.48,0.48,0.64,0.56,0.48,0.48
9,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.60,0.56,0.52,0.64,0.56,0.56,0.52,0.52,0.60,0.52


In [ ]:
# Initialize the DRL model
input_dim = 52  # Since each hand has 26 cards, so the total state dimension is 52
output_dim = 4  # Assuming 4 possible actions (contracts)
model = create_model(input_dim, output_dim)

# Train the model
train_drl_model(data, model, epochs=100)

# Example: Testing the model on a new hand
test_hand = np.random.rand(26)
test_state = np.array([test_hand, test_hand]).reshape(1, -1)
predicted_q_values = model.predict(test_state)
predicted_bid = np.argmax(predicted_q_values[0])  # The predicted best contract (bid)
print(f"Predicted best contract bid: {predicted_bid}")